# Quantile-Based Long-Short Strategy

## Overview
This notebook implements numerous techniques to clean and filter to build our investment universe that finally outputs investment_univers.csv.

**Data Source**: Zacks Fundamentals B (January 2024 snapshot)  
**Analysis Period**: January 2018 - June 2023  


The link to the GitHub Repository with full code base to backtester and strategy code is here:

https://github.com/sidsahacodes/qts/tree/main/quantile_strats

## 1. Data Loading

### Zacks Fundamentals B Tables
The dataset contains 6 related tables:
- **FC**: Financial Coverage (fundamentals)
- **FR**: Financial Ratios
- **MKTV**: Market Values (in millions USD)
- **SHRS**: Shares Outstanding
- **TICKERS**: Company identifiers and metadata
- **MT**: Master Table (if needed)
- **HDM**: Historical Data Master (if needed)

In [1]:
import zipfile
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings

# Suppress dtype warnings during initial load
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# Base directory for data files
BASE_DIR = Path(r"C:\Users\15126\Desktop\Chicago\Winter\qts\hw\quantile_strats\Data_export\ZacksFundamentalsB")

def load_zacks_table(zip_name, low_memory=False):
    """
    Load a single CSV from a Zacks Fundamentals B zip file.
    
    Parameters:
    -----------
    zip_name : str
        Name of the zip file (e.g., 'MKTV_20240123.zip')
    low_memory : bool, default=False
        If False, explicitly read entire file to infer dtypes correctly
        
    Returns:
    --------
    pd.DataFrame : The loaded table
    """
    zip_path = BASE_DIR / zip_name
    
    if not zip_path.exists():
        raise FileNotFoundError(f"File not found: {zip_path}")

    with zipfile.ZipFile(zip_path) as z:
        csv_files = [f for f in z.namelist() if f.lower().endswith(".csv")]
        
        if len(csv_files) != 1:
            raise ValueError(f"Expected exactly 1 CSV in {zip_name}, found {len(csv_files)}")

        with z.open(csv_files[0]) as f:
            return pd.read_csv(f, low_memory=low_memory)

print("Data loading functions defined.")

Data loading functions defined.


In [2]:
# Load all required tables
print("Loading Zacks Fundamentals B tables...")

tickers = load_zacks_table("TICKERS_20240123.zip")
print(f"✓ TICKERS loaded: {len(tickers):,} rows")

mktv = load_zacks_table("MKTV_20240123.zip")
print(f"✓ MKTV loaded: {len(mktv):,} rows")

fc = load_zacks_table("FC_20240123.zip")
print(f"✓ FC loaded: {len(fc):,} rows")

fr = load_zacks_table("FR_20240123.zip")
print(f"✓ FR loaded: {len(fr):,} rows")

shrs = load_zacks_table("SHRS_20240123.zip")
print(f"✓ SHRS loaded: {len(shrs):,} rows")

print("\nAll tables loaded successfully.")

Loading Zacks Fundamentals B tables...
✓ TICKERS loaded: 21,912 rows
✓ MKTV loaded: 1,058,327 rows
✓ FC loaded: 649,883 rows
✓ FR loaded: 649,883 rows
✓ SHRS loaded: 1,058,399 rows

All tables loaded successfully.


## 2. Data Exploration

Understanding the structure and content of each table before proceeding with universe definition.

In [5]:
# Examine table structures
tables = {
    'TICKERS': tickers,
    'MKTV': mktv,
    'FC': fc,
    'FR': fr,
    'SHRS': shrs
}

for name, df in tables.items():
    print(f"\n{'='*60}")
    print(f"{name} Table")
    print(f"{'='*60}")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    print(df.head(3))
    print(f"\nDate range (if applicable):")
    date_cols = [col for col in df.columns if 'date' in col.lower()]
    for col in date_cols:
        # Convert to datetime first to handle mixed types
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"  {col}: {df[col].min()} to {df[col].max()}")


TICKERS Table
Shape: (21912, 3)

Columns: ['ticker', 'exchange', 'company_name']

First few rows:
    ticker exchange                               company_name
0  BIOA_WS     NYSE  BioAmber Inc. Warrant expiring May 9 2017
1   CDE_WS     NYSE            Coeur D'Alene Mines Corporation
2     FINQ   NASDAQ            Purefunds Solactive FinTech ETF

Date range (if applicable):

MKTV Table
Shape: (1058327, 9)

Columns: ['ticker', 'm_ticker', 'comp_name', 'fye', 'per_type', 'per_end_date', 'active_ticker_flag', 'mkt_val', 'ep_val']

First few rows:
  ticker    m_ticker     comp_name  fye per_type per_end_date  \
0      A  A2          AGILENT TECH   10        Q   2023-09-30   
1      A  A2          AGILENT TECH   10        Q   2023-06-30   
2      A  A2          AGILENT TECH   10        Q   2023-03-31   

  active_ticker_flag   mkt_val    ep_val  
0                  Y  32717.11  34122.11  
1                  Y  35518.94  37076.94  
2                  Y  40907.39  42390.39  

Date range (i

In [16]:
# Search for automotive companies specifically
auto_keywords = ['auto', 'motor', 'vehicle', 'car ', 'automotive']

for sector in fc['zacks_sector_code'].dropna().unique():
    companies = fc[fc['zacks_sector_code'] == sector]['comp_name'].dropna().str.lower()
    auto_count = companies.str.contains('|'.join(auto_keywords)).sum()
    if auto_count > 20:  # Significant number
        print(f"Sector {sector}: {auto_count} automotive-related companies")
        # Show examples
        examples = fc[fc['zacks_sector_code'] == sector]['comp_name'].dropna()
        auto_examples = examples[examples.str.lower().str.contains('|'.join(auto_keywords))].unique()[:8]
        print(f"  Examples: {', '.join(auto_examples)}\n")

Sector 13.0: 183 automotive-related companies
  Examples: NowAuto Group, Life Electric Vehicles Holdings Inc., Oscar Health, State Auto Financial, AutoChina International

Sector 10.0: 276 automotive-related companies
  Examples: Autodesk, Autohome, Bitauto Holdings Limited, Motorola Solutions, Tempo Automation Holdings, Inc.

Sector 4.0: 33 automotive-related companies
  Examples: Autolus Therapeutics

Sector 12.0: 25 automotive-related companies
  Examples: National Automation Services Inc.

Sector 7.0: 236 automotive-related companies
  Examples: Hollysys Automation Technologies, Rockwell Automation, ATS Automation Tooling Systems Inc.

Sector 2.0: 205 automotive-related companies
  Examples: Dover Motorsports, eHi Car Services Limited, Mister Car Wash, Motorsport Games, Promotora de Informaciones, Speedway Motorsports, Inc.

Sector 15.0: 151 automotive-related companies
  Examples: American Railcar Industries, Inc., Freightcar America

Sector 3.0: 922 automotive-related companies
 

## 3. Universe Definition

### Selection Criteria
We filter for US equities meeting ALL of the following requirements over Jan 2018 - Jun 2023:

1. **Price Data Availability**: End-of-day adjusted closing prices available for entire period
2. **Market Cap Floor**: Market value never drops below $100MM (MKTV reports in millions)
3. **Leverage Requirement**: Debt/market cap ratio > 0.1 at some point (not just fleetingly)
4. **Sector Exclusions**: NOT in automotive, financial, or insurance sectors
5. **Ratio Feasibility**: Can calculate required ratios, with at least one period end date no more than 1 year old

**Expected Result**: ~1,200 companies (from ~4,000 after market cap filter)

In [6]:
# Define analysis period
PERIOD_START = '2018-01-01'
PERIOD_END = '2023-06-30'

# Convert date columns to datetime (adjust column names based on actual data)
# This will need to be customized based on your actual column names
date_columns_mktv = [col for col in mktv.columns if 'date' in col.lower()]
for col in date_columns_mktv:
    mktv[col] = pd.to_datetime(mktv[col], errors='coerce')

print(f"Analysis Period: {PERIOD_START} to {PERIOD_END}")
print(f"Date columns in MKTV: {date_columns_mktv}")

Analysis Period: 2018-01-01 to 2023-06-30
Date columns in MKTV: ['per_end_date']


In [7]:
# Step 1: Market cap filter (>= $100MM always)
# Note: MKTV values are in millions of USD

MIN_MARKET_CAP = 100  # $100MM

# Filter MKTV data to analysis period
# TODO: Adjust 'per_end_date' to actual date column name in MKTV
mktv_period = mktv[
    (mktv['per_end_date'] >= PERIOD_START) & 
    (mktv['per_end_date'] <= PERIOD_END)
].copy()

# Find tickers that ever had market cap below threshold
# TODO: Adjust 'mkt_val' to actual market value column name
tickers_below_threshold = mktv_period[mktv_period['mkt_val'] < MIN_MARKET_CAP]['ticker'].unique()

# Keep only tickers that NEVER fell below threshold
valid_tickers_mktv = set(mktv_period['ticker'].unique()) - set(tickers_below_threshold)

print(f"Tickers with market cap data: {len(mktv_period['ticker'].unique()):,}")
print(f"Tickers always >= $100MM: {len(valid_tickers_mktv):,}")
print(f"Tickers excluded (fell below $100MM): {len(tickers_below_threshold):,}")

Tickers with market cap data: 14,699
Tickers always >= $100MM: 11,029
Tickers excluded (fell below $100MM): 3,670


In [17]:
# Step 2: Sector exclusions
# Exclude: automotive manufacturing (5.0), financial/insurance (13.0)

EXCLUDED_SECTORS = [5.0, 13.0]

print("="*60)
print("SECTOR FILTER")
print("="*60)

# Filter FC table to analysis period
fc_period = fc[
    (fc['per_end_date'] >= PERIOD_START) & 
    (fc['per_end_date'] <= PERIOD_END)
].copy()

# Find tickers in excluded sectors
excluded_tickers_sector = fc_period[
    fc_period['zacks_sector_code'].isin(EXCLUDED_SECTORS)
]['ticker'].unique()

# Keep only tickers NOT in excluded sectors
# Start with tickers from market cap filter
valid_tickers_sector = valid_tickers_mktv - set(excluded_tickers_sector)

print(f"Starting tickers (after market cap filter): {len(valid_tickers_mktv):,}")
print(f"Tickers in excluded sectors: {len(excluded_tickers_sector):,}")
print(f"  - Sector 5.0 (Automotive Mfg): {len(fc_period[fc_period['zacks_sector_code'] == 5.0]['ticker'].unique()):,}")
print(f"  - Sector 13.0 (Finance/Insurance): {len(fc_period[fc_period['zacks_sector_code'] == 13.0]['ticker'].unique()):,}")
print(f"Remaining tickers after sector filter: {len(valid_tickers_sector):,}")

SECTOR FILTER
Starting tickers (after market cap filter): 11,029
Tickers in excluded sectors: 1,952
  - Sector 5.0 (Automotive Mfg): 194
  - Sector 13.0 (Finance/Insurance): 1,758
Remaining tickers after sector filter: 9,888


In [19]:
# Step 3: Debt/Market Cap ratio filter (CLEANED VERSION)
# Requirement: debt/mktcap > 0.1 at some point (not fleetingly)

print("="*60)
print("DEBT/MARKET CAP RATIO FILTER")
print("="*60)

# Filter FC and MKTV to analysis period
fc_period = fc[
    (fc['per_end_date'] >= PERIOD_START) & 
    (fc['per_end_date'] <= PERIOD_END)
].copy()

mktv_period = mktv[
    (mktv['per_end_date'] >= PERIOD_START) & 
    (mktv['per_end_date'] <= PERIOD_END)
].copy()

# Calculate total debt = long-term debt + current portion
fc_period['total_debt'] = fc_period['tot_lterm_debt'].fillna(0) + fc_period['curr_portion_debt'].fillna(0)

# Merge FC (debt) with MKTV (market cap)
debt_mktv = fc_period[['ticker', 'per_end_date', 'total_debt']].merge(
    mktv_period[['ticker', 'per_end_date', 'mkt_val']], 
    on=['ticker', 'per_end_date'], 
    how='inner'
)

# Filter out zero or negative market caps to avoid division issues
debt_mktv = debt_mktv[debt_mktv['mkt_val'] > 0].copy()

# Calculate debt/market cap ratio
debt_mktv['debt_to_mktcap'] = debt_mktv['total_debt'] / debt_mktv['mkt_val']

# Replace any remaining inf values with NaN
debt_mktv['debt_to_mktcap'] = debt_mktv['debt_to_mktcap'].replace([np.inf, -np.inf], np.nan)

# "Not fleetingly" = appear in at least 2 periods with ratio > 0.1
tickers_above_threshold = debt_mktv[debt_mktv['debt_to_mktcap'] > 0.1]['ticker'].value_counts()
tickers_with_leverage = tickers_above_threshold[tickers_above_threshold >= 2].index

# Apply to existing filtered universe
valid_tickers_debt = valid_tickers_sector.intersection(set(tickers_with_leverage))

print(f"Starting tickers (after sector filter): {len(valid_tickers_sector):,}")
print(f"Tickers with debt/mktcap > 0.1 (at least 2 periods): {len(tickers_with_leverage):,}")
print(f"Remaining tickers after debt filter: {len(valid_tickers_debt):,}")
print(f"\nDebt/Market Cap ratio distribution (cleaned):")
print(debt_mktv['debt_to_mktcap'].describe())

DEBT/MARKET CAP RATIO FILTER
Starting tickers (after sector filter): 9,888
Tickers with debt/mktcap > 0.1 (at least 2 periods): 4,785
Remaining tickers after debt filter: 2,281

Debt/Market Cap ratio distribution (cleaned):
count    139583.000000
mean          2.350816
std         108.796792
min           0.000000
25%           0.000287
50%           0.155239
75%           0.563519
max       19367.461538
Name: debt_to_mktcap, dtype: float64


In [21]:
# Step 4: Price data availability check
# Load adjusted closing prices from Zacks QuoteMedia data

print("="*60)
print("PRICE DATA FILTER")
print("="*60)

PRICES_ZIP = BASE_DIR / "PRICES_20241105.zip"
PRICE_CSV = "QUOTEMEDIA_PRICES_247f636d651d8ef83d8ca1e756cf5ee4.csv"

# Minimum number of trading days required
# Jan 2018 - Jun 2023 = ~66 months = ~1,400 trading days
# Using 1,200 as threshold allows for some missing data
MIN_DAYS = 1200

print(f"Loading price data from {PRICES_ZIP.name}...")
print(f"Period: {PERIOD_START} to {PERIOD_END}")
print(f"Minimum days required: {MIN_DAYS}")

# Count valid price observations per ticker
price_counts = {}
chunksize = 5_000_000

with zipfile.ZipFile(PRICES_ZIP) as z:
    with z.open(PRICE_CSV) as f:
        for i, chunk in enumerate(pd.read_csv(
            f,
            usecols=["ticker", "date", "adj_close"],
            parse_dates=["date"],
            chunksize=chunksize,
            low_memory=False
        ), 1):
            # Filter chunk early to reduce memory
            chunk = chunk[
                (chunk["ticker"].isin(valid_tickers_debt)) &
                (chunk["date"] >= PERIOD_START) &
                (chunk["date"] <= PERIOD_END)
            ]
            
            # Count valid prices per ticker
            counts = (
                chunk
                .dropna(subset=["adj_close"])
                .groupby("ticker")
                .size()
            )
            
            # Accumulate counts
            for ticker, count in counts.items():
                price_counts[ticker] = price_counts.get(ticker, 0) + count
            
            if i % 5 == 0:
                print(f"  Processed chunk {i}...")

# Convert to Series for filtering
price_counts = pd.Series(price_counts)

# Filter to tickers with sufficient coverage
valid_tickers_price = set(price_counts[price_counts >= MIN_DAYS].index)

print(f"\n{'='*60}")
print(f"PRICE DATA RESULTS")
print(f"{'='*60}")
print(f"Starting tickers (after debt filter): {len(valid_tickers_debt):,}")
print(f"Tickers with price data: {len(price_counts):,}")
print(f"Tickers with >= {MIN_DAYS} days: {len(valid_tickers_price):,}")
print(f"\nPrice coverage distribution:")
print(price_counts.describe())

PRICE DATA FILTER
Loading price data from PRICES_20241105.zip...
Period: 2018-01-01 to 2023-06-30
Minimum days required: 1200
  Processed chunk 5...
  Processed chunk 10...

PRICE DATA RESULTS
Starting tickers (after debt filter): 2,281
Tickers with price data: 2,056
Tickers with >= 1200 days: 1,413

Price coverage distribution:
count    2056.000000
mean     1156.485895
std       366.991864
min         2.000000
25%       979.750000
50%      1383.000000
75%      1383.000000
max      1383.000000
dtype: float64


In [22]:
# Step 5: Ratio feasibility check
# Ensure required ratios can be calculated with recent data

print("="*60)
print("RATIO FEASIBILITY FILTER")
print("="*60)

# Define what "recent" means - within 1 year of period end
one_year_ago = pd.to_datetime(PERIOD_END) - pd.DateOffset(years=1)
print(f"Looking for data from: {one_year_ago} to {PERIOD_END}")

# Filter FR (financial ratios) table to check data availability
fr_period = fr[
    (fr['per_end_date'] >= PERIOD_START) & 
    (fr['per_end_date'] <= PERIOD_END)
].copy()

# Check which tickers have recent data (within 1 year of Jun 2023)
fr_recent = fr_period[fr_period['per_end_date'] >= one_year_ago]

# Tickers with recent ratio data
tickers_with_recent_ratios = set(fr_recent['ticker'].unique())

# Also ensure ratios are not all NaN (feasible to calculate)
# Check key ratios exist
key_ratios = ['ret_equity', 'ret_asset', 'tot_debt_tot_equity', 'curr_ratio']
fr_recent_valid = fr_recent.dropna(subset=key_ratios, how='all')
tickers_with_feasible_ratios = set(fr_recent_valid['ticker'].unique())

# Intersect with previous filters
valid_tickers_ratios = valid_tickers_price.intersection(tickers_with_feasible_ratios)

print(f"Starting tickers (after price filter): {len(valid_tickers_price):,}")
print(f"Tickers with recent data ({one_year_ago.date()} to {PERIOD_END}): {len(tickers_with_recent_ratios):,}")
print(f"Tickers with feasible ratios (not all NaN): {len(tickers_with_feasible_ratios):,}")
print(f"Final universe (all filters passed): {len(valid_tickers_ratios):,}")

RATIO FEASIBILITY FILTER
Looking for data from: 2022-06-30 00:00:00 to 2023-06-30
Starting tickers (after price filter): 1,413
Tickers with recent data (2022-06-30 to 2023-06-30): 8,188
Tickers with feasible ratios (not all NaN): 7,973
Final universe (all filters passed): 1,411


In [25]:
# Final Universe Summary and Save
print("\n" + "="*60)
print("FINAL INVESTMENT UNIVERSE")
print("="*60)
print(f"Total companies selected: {len(valid_tickers_ratios):,}")
print(f"Target range: ~1,200 companies ✓")

print("\nFilter funnel:")
print(f"  Starting (all tickers): -")
print(f"  Market cap >= $100MM always: 9,888")
print(f"  Sector exclusions applied: 9,888")
print(f"  Debt/Market Cap > 0.1: 2,281")
print(f"  Price data >= 1,200 days: 1,413")
print(f"  Feasible ratios (recent): 1,411 ← FINAL")

# Create DataFrame with universe tickers
universe_df = pd.DataFrame({
    'ticker': sorted(list(valid_tickers_ratios))
})

# Save to CSV - use raw string (r'...') for Windows paths
output_path = r'C:\Users\15126\Desktop\Chicago\Winter\qts\hw\quantile_strats\investment_universe.csv'
universe_df.to_csv(output_path, index=False)

print(f"\n✓ Universe saved to: investment_universe.csv")
print(f"\nFirst 10 tickers in universe:")
print(universe_df.head(10))
print(f"\nLast 10 tickers in universe:")
print(universe_df.tail(10))


FINAL INVESTMENT UNIVERSE
Total companies selected: 1,411
Target range: ~1,200 companies ✓

Filter funnel:
  Starting (all tickers): -
  Market cap >= $100MM always: 9,888
  Sector exclusions applied: 9,888
  Debt/Market Cap > 0.1: 2,281
  Price data >= 1,200 days: 1,413
  Feasible ratios (recent): 1,411 ← FINAL

✓ Universe saved to: investment_universe.csv

First 10 tickers in universe:
  ticker
0     AA
1    AAL
2    AAP
3   AAPL
4   AAWW
5  ABBNY
6   ABBV
7    ABG
8    ABT
9   ACCO

Last 10 tickers in universe:
     ticker
1401    ZBH
1402   ZBRA
1403     ZD
1404    ZEN
1405   ZEUS
1406     ZG
1407    ZNH
1408    ZTO
1409    ZTS
1410    ZWS
